In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Additional imports for ROC, AUC, and Confusion Matrix
from sklearn.preprocessing import label_binarize
from itertools import cycle

# Combined ResNet and MobileNet model with Attention, Dropout, and Regularization
class CombinedModel(nn.Module):
    def __init__(self, num_classes):
        super(CombinedModel, self).__init__()

        # Load pre-trained ResNet (from torchvision)
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.resnet_features = nn.Sequential(*list(self.resnet.children())[:-2])  # Remove final FC and AvgPool layer

        # Load pre-trained MobileNetV3 from torchvision
        self.mobilenet = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
        self.mobilenet_features = nn.Sequential(*list(self.mobilenet.children())[:-1])  # Remove classifier

        # Global Average Pooling (GAP) after feature extraction
        self.gap = nn.AdaptiveAvgPool2d(1)  # Adaptive pooling to (1, 1)

        # Calculate the actual combined feature size dynamically based on the models
        self.resnet_feature_size = self.resnet.fc.in_features  # ResNet's last layer in_features size
        self.mobilenet_feature_size = self.mobilenet.classifier[0].in_features  # MobileNet's classifier in_features
        self.combined_feature_size = self.resnet_feature_size + self.mobilenet_feature_size

        # Attention layer for combined features
        self.attention = AttentionLayer(self.combined_feature_size)

        # Classification head (with dropout to avoid overfitting)
        self.classifier = nn.Sequential(
            nn.Linear(self.combined_feature_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),  # Increase dropout for regularization
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),  # Another dropout layer
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # Extract features from ResNet
        resnet_features = self.resnet_features(x)
        resnet_features = self.gap(resnet_features)  # Apply GAP
        resnet_features = resnet_features.view(resnet_features.size(0), -1)

        # Extract features from MobileNet
        mobilenet_features = self.mobilenet_features(x)
        mobilenet_features = self.gap(mobilenet_features)  # Apply GAP
        mobilenet_features = mobilenet_features.view(mobilenet_features.size(0), -1)

        # Concatenate the feature maps
        combined_features = torch.cat((resnet_features, mobilenet_features), dim=1)

        # Pass through attention mechanism
        combined_features = self.attention(combined_features)

        # Classification head
        out = self.classifier(combined_features)
        return out

# Data Augmentation: Enhanced with random cropping, color jitter, etc.
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Random crop and resize to 224x224
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),  # Random color jitter
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for consistency
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load dataset using ImageFolder
data_dir = '/content/drive/MyDrive/clothing_data'
dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)

# Split the dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model Checkpointing (save the model with the lowest validation loss)
def save_model(model, optimizer, epoch, loss, path='best_model.pth'):
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss
    }
    torch.save(state, path)

# Training and Evaluation Functions
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}')

        # Validate model after each epoch
        avg_val_loss = validate_model(model, test_loader, criterion)
        print(f'Epoch [{epoch+1}/{num_epochs}], Val Loss: {avg_val_loss:.4f}')

        # Save the model with the lowest validation loss
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            save_model(model, optimizer, epoch, best_loss)
            print(f"Model saved at epoch {epoch+1} with validation loss {best_loss:.4f}")

def validate_model(model, test_loader, criterion):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    return val_loss / len(test_loader)

def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy * 100:.2f}%')

# Instantiate the model, criterion, and optimizer
num_classes = 8  # Adjust this based on your dataset
model = CombinedModel(num_classes).cuda()

# Use CrossEntropyLoss and Adam optimizer with weight decay (L2 regularization)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Train and Evaluate the Model
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10)

# Load the best saved model for final evaluation
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint['model'])
evaluate_model(model, test_loader)

# Evaluation function to include AUC, ROC, Classification Report, and Confusion Matrix
def evaluate_model(model, test_loader, num_classes):
    model.eval()
    y_true = []
    y_pred = []
    y_prob = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.cuda(), labels.cuda()

            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)  # Get probabilities for AUC/ROC

            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_prob.extend(probabilities.cpu().numpy())

    # Classification Report
    print("Classification Report:\n")
    print(classification_report(y_true, y_pred))

    # Confusion Matrix
    print("Confusion Matrix:\n")
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(num_classes), yticklabels=range(num_classes))
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    # ROC Curve and AUC for each class
    y_true_bin = label_binarize(y_true, classes=range(num_classes))  # Binarize labels for multi-class ROC
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], np.array(y_prob)[:, i])
        roc_auc[i] = roc_auc_score(y_true_bin[:, i], np.array(y_prob)[:, i])

    # Plot ROC curve for each class
    plt.figure()
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green', 'purple', 'brown', 'pink', 'gray', 'yellow'])
    for i, color in zip(range(num_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Class {i} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

    # AUC Score (micro-average)
    print(f'AUC (micro-average): {roc_auc_score(y_true_bin, y_prob, average="micro"):.4f}')

# Train and Validate as before (without changes)

# Load the best saved model for final evaluation
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint['model'])

# Evaluate the model with AUC, ROC, and Confusion Matrix
evaluate_model(model, test_loader, num_classes=num_classes)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small, EfficientNetV2B0, ResNet50, DenseNet121
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as efficientnet_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
data_dir = "/content/drive/MyDrive/Clothing_Data_HRP"

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

In [ ]:
# Load Data and Split with class balance
def load_data(data_dir):
    data_generator = ImageDataGenerator(rescale=1./255)
    data_flow = data_generator.flow_from_directory(
        data_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )
    X, y = [], []
    for _ in range(len(data_flow)):
        x_batch, y_batch = next(data_flow)
        X.extend(x_batch)
        y.extend(y_batch)
    return np.array(X), np.array(y)

In [ ]:
# Split the data into train and validation sets with stratification
X, y = load_data(data_dir)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Found 810 images belonging to 8 classes.


In [ ]:
X_train.shape

(648, 224, 224, 3)

In [ ]:
# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(np.argmax(y_train, axis=1)),
    y=np.argmax(y_train, axis=1)
)
class_weights = dict(enumerate(class_weights))

In [ ]:
class_weights

{0: 1.0125,
 1: 1.0125,
 2: 1.0125,
 3: 1.0125,
 4: 1.0125,
 5: 1.0125,
 6: 1.0125,
 7: 0.9204545454545454}

In [ ]:
# Define callbacks: EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

In [ ]:
# Function to build models
def build_model(model_name, preprocess_func):
    base_model = None
    if model_name == 'mobilenet_v3_small':
        base_model = MobileNetV3Small(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
    elif model_name == 'efficientnet_v2_b0':
        base_model = EfficientNetV2B0(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
    elif model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
    elif model_name == 'densenet121':
        base_model = DenseNet121(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

    base_model.trainable = False

    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
        tf.keras.layers.Lambda(preprocess_func),
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(np.unique(np.argmax(y_train, axis=1))), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Model List
models = {
    'mobilenet_v3_small': mobilenet_preprocess,
    'efficientnet_v2_b0': efficientnet_preprocess,
    'resnet50': resnet_preprocess,
    'densenet121': densenet_preprocess
}

In [ ]:
# Train each model and evaluate
best_model = None
best_accuracy = 0
history = None

for model_name, preprocess_func in models.items():
    print(f'Training {model_name}...')
    model = build_model(model_name, preprocess_func)

    hist = model.fit(
        train_datagen.flow(X_train, y_train, batch_size=32),
        validation_data=val_datagen.flow(X_val, y_val),
        epochs=50,
        callbacks=[early_stop, checkpoint],
        class_weight=class_weights,
        verbose=1
    )

    # Evaluate on validation data
    val_loss, val_acc = model.evaluate(val_datagen.flow(X_val, y_val), verbose=0)

    # Save the best model
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        best_model = model
        history = hist

Training mobilenet_v3_small...
4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.1103 - loss: 2.1735 - val_accuracy: 0.1049 - val_loss: 2.1222
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 218ms/step - accuracy: 0.1239 - loss: 2.1600 - val_accuracy: 0.1049 - val_loss: 2.0833
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 212ms/step - accuracy: 0.1050 - loss: 2.1277 - val_accuracy: 0.1235 - val_loss: 2.0897
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 299ms/step - accuracy: 0.1356 - loss: 2.0995 - val_accuracy: 0.1235 - val_loss: 2.0905
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 376ms/step - accuracy: 0.1301 - loss: 2.1045 - val_accuracy: 0.1296 - val_loss: 2.0809
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 242ms/step - accuracy: 0.1272 - loss: 2.1057 - val_accuracy: 0.1358 - val_loss: 2.0802
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 317ms/step - accuracy: 0.1283 - loss: 2.0990 - val_accuracy: 0.1235 - val_loss: 2.0824
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 199ms/step - accuracy: 0.1460 - loss: 2.0764 - val_accuracy: 0.1235 - val_

In [ ]:
# Load the best model
best_model.load_weights('best_model.keras')

ValueError: A total of 102 objects could not be loaded. Example error message for object <Conv2D name=conv1_conv, built=True>:

Layer 'conv1_conv' expected 2 variables, but received 1 variables during loading. Expected: ['kernel', 'bias']

List of objects that could not be loaded:
[<Conv2D name=conv1_conv, built=True>, <Conv2D name=conv2_block1_1_conv, built=True>, <Conv2D name=conv2_block1_2_conv, built=True>, <BatchNormalization name=conv2_block1_2_bn, built=True>, <Conv2D name=conv2_block1_0_conv, built=True>, <Conv2D name=conv2_block1_3_conv, built=True>, <BatchNormalization name=conv2_block1_0_bn, built=True>, <BatchNormalization name=conv2_block1_3_bn, built=True>, <Conv2D name=conv2_block2_1_conv, built=True>, <BatchNormalization name=conv2_block2_1_bn, built=True>, <Conv2D name=conv2_block2_2_conv, built=True>, <BatchNormalization name=conv2_block2_2_bn, built=True>, <Conv2D name=conv2_block2_3_conv, built=True>, <BatchNormalization name=conv2_block2_3_bn, built=True>, <Conv2D 

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transforms with advanced augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Clothing_Data_HRP', transform=train_transform)

# Calculate class weights for balanced sampling
class_weights = []
for root, subdir, files in os.walk('/content/drive/MyDrive/Clothing_Data_HRP'):
    if len(files) > 0:
        class_weights.append(1/len(files))
sample_weights = [0] * len(dataset)
for idx, (data, label) in enumerate(dataset):
    class_weight = class_weights[label]
    sample_weights[idx] = class_weight

# Create weighted sampler
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# Split the dataset
train_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, stratify=dataset.targets)

train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(datasets.ImageFolder(root='clothing', transform=test_transform), test_indices)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define models
model_functions = {
    'mobilenet_v3_small': models.mobilenet_v3_small,
    'efficientnet_v2_b0': models.efficientnet_v2_s,
    'resnet50': models.resnet50,
    'regnet_y_400mf': models.regnet_y_400mf,
    'densenet121': models.densenet121
}

def train_model(model_name, num_classes, num_epochs=50, patience=10):
    model = model_functions[model_name](pretrained=True)

    # Modify the last layer for our number of classes
    if model_name.startswith('mobilenet') or model_name.startswith('efficientnet'):
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)
    elif model_name.startswith('resnet') or model_name.startswith('regnet'):
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name.startswith('densenet'):
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)

    best_acc = 0.0
    best_model = None
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_acc = val_correct / val_total
        scheduler.step(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            best_model = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print("Early stopping")
                break

    return best_model, best_acc

# Train and evaluate each model
results = {}
for model_name in model_functions.keys():
    print(f"Training {model_name}")
    best_model, best_acc = train_model(model_name, len(dataset.classes))
    results[model_name] = (best_model, best_acc)

# Find the best performing model
best_model_name = max(results, key=lambda k: results[k][1])
best_model_state, best_acc = results[best_model_name]

print(f"Best model: {best_model_name} with accuracy: {best_acc:.4f}")

# Save the best model
torch.save(best_model_state, f"best_model_{best_model_name}.pth")

# Load the best model for evaluation
model = model_functions[best_model_name](pretrained=False)
if best_model_name.startswith('mobilenet') or best_model_name.startswith('efficientnet'):
    model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, len(dataset.classes))
elif best_model_name.startswith('resnet') or best_model_name.startswith('regnet'):
    model.fc = nn.Linear(model.fc.in_features, len(dataset.classes))
elif best_model_name.startswith('densenet'):
    model.classifier = nn.Linear(model.classifier.in_features, len(dataset.classes))

model.load_state_dict(best_model_state)
model = model.to(device)
model.eval()

# Evaluation
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=dataset.classes))

print("\nConfusion Matrix:")
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(dataset.classes))
plt.xticks(tick_marks, dataset.classes, rotation=45)
plt.yticks(tick_marks, dataset.classes)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig('confusion_matrix.png')
plt.close()

print(f"Confusion matrix saved as 'confusion_matrix.png'")

FileNotFoundError: [Errno 2] No such file or directory: 'clothing'